# **Predict Future Sales**

> # **Импорт библиотек**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import time
import re

from math import sqrt
from numpy import loadtxt
from itertools import product
from tqdm import tqdm
from sklearn import preprocessing
from xgboost import plot_tree
from matplotlib import pyplot

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

> # **Чтение данных**

In [ ]:
sales_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
item_categories = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
sample_submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')

> # **Отображение информации о каждом датасете**

In [ ]:
print('---------info----------')
sales_train.info()
print('---------nunique----------')
sales_train.nunique()

In [ ]:
print('---------info----------')
items.info()
print('---------nunique----------')
items.nunique()

In [ ]:
print('---------info----------')
item_categories.info()
print('---------nunique----------')
item_categories.nunique()

In [ ]:
print('---------info----------')
shops.info()
print('---------nunique----------')
shops.nunique()

> # **Первичный анализ данных и визуализация данных**

In [ ]:
plt.figure(figsize=(40, 10))
plt.title('Сумма продаж по магазинам', fontdict={'fontsize': 40})
plt.xlabel(xlabel='Магазин', fontdict={'fontsize': 30})
plt.ylabel(ylabel='Продажи', fontdict={'fontsize': 30})
sns.countplot(x='shop_id', data=sales_train)

In [ ]:
plt.figure(figsize=(40, 10))
plt.title('Сумма продаж по месяцам', fontdict={'fontsize': 40})
plt.xlabel(xlabel='Месяц', fontdict={'fontsize': 30})
plt.ylabel(ylabel='Продажи', fontdict={'fontsize': 30})
sns.countplot(x='date_block_num', data=sales_train)

**Распределение целевой переменной:**

In [ ]:
sales_train.boxplot(column=['item_cnt_day'], grid = True, figsize = (10, 10))

In [ ]:
sales_train.boxplot(column=['item_price'], grid = True, figsize = (10, 10))

In [ ]:
sales_train = sales_train[(sales_train.item_price <= 50000) & (sales_train.item_cnt_day <= 1000)]
sales_train = sales_train[(sales_train.item_price > 0) & (sales_train.item_cnt_day > 0)].reset_index(drop = True)

In [ ]:
sales_train.boxplot(column=['item_cnt_day'], grid = True, figsize = (10, 10))

In [ ]:
sales_train.boxplot(column=['item_price'], grid = True, figsize = (10, 10))

In [ ]:
# Якутск Орджоникидзе, 56
sales_train.loc[sales_train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57

#Якутск ТЦ "Центральный"
sales_train.loc[sales_train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58

# Жуковский ул. Чкалова 39м²
sales_train.loc[sales_train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

In [ ]:
shops['city'] = shops.shop_name.str.split(' ').map(lambda x: x[0])
shops['category'] = shops.shop_name.str.split(' ').map(lambda x: x[1] if len(x[1]) <= 5 else 'Другое')
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops.head()

In [ ]:
lb = preprocessing.LabelEncoder()
shops['shop_category'] = lb.fit_transform(shops.category)
shops['shop_city'] = lb.fit_transform(shops.city)
shops.head()

In [ ]:
item_ctgrs = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
item_ctgrs['type_category'] = item_ctgrs.item_category_name.apply(lambda x: x.split(' ')[0]).astype(str)

item_ctgrs.loc[(item_ctgrs.type_category == 'Доставка'), 'type_category'] = 'Доставка товара'
item_ctgrs.loc[(item_ctgrs.type_category == 'Игровые'), 'type_category'] = 'Игровые консоли'
item_ctgrs.loc[(item_ctgrs.item_category_name == 'Игры Android'), 'type_category'] = 'Игры Android'
item_ctgrs.loc[(item_ctgrs.item_category_name == 'Игры MAC'), 'type_category'] = 'Игры MAC'
item_ctgrs.loc[(item_ctgrs.item_category_name == 'Игры PC'), 'type_category'] = 'Игры PC'
item_ctgrs.loc[(item_ctgrs.type_category == 'Карты'), 'type_category'] = 'Карты оплаты'
item_ctgrs.loc[(item_ctgrs.type_category == 'Чистые'), 'type_category'] = 'Чистые носители'
item_ctgrs.loc[(item_ctgrs.type_category == 'Элементы'), 'type_category'] = 'Элементы питания'

item_ctgrs['type_category_id'] = lb.fit_transform(item_ctgrs.type_category)

item_ctgrs.head(60)

> # **Создание датафрейма, содержащего все комбинации shop_id, item_id, date_block_num**

In [ ]:
grid = []
for date_block_num in sales_train['date_block_num'].unique():
    cur_shops = sales_train[sales_train['date_block_num'] == date_block_num]['shop_id'].unique()
    cur_items = sales_train[sales_train['date_block_num'] == date_block_num]['item_id'].unique()
    grid.append(np.array(list(product(cur_shops, cur_items, [date_block_num])), dtype = 'int32'))
index_cols = ['shop_id', 'item_id', 'date_block_num']
grid = pd.DataFrame(np.vstack(grid), columns = index_cols, dtype = np.int32)

In [ ]:
grid.head()

**sales_train содержит целевую переменную item_cnt_day - количество продаж в день. Изменим целевую переменную на item_cnt_month - количество продаж в месяц. Возьмем сумму продаж в месяц по каждому товару в кажом магазине и среднюю стоимость данного товара. С целью нормализации распределения целевой переменной зададим количества продаж от 0 до 30 при этом все продажи, превосходящие 30, буду принимать значение 30**

In [ ]:
sales_train['item_cnt_day'] = sales_train['item_cnt_day'].clip(0, 30)
groups = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'])
train_set = groups.agg({'item_cnt_day':'sum', 'item_price':'mean'}).reset_index()
train_set = train_set.rename(columns = {'item_cnt_day':'item_cnt_month'})
train_set['item_cnt_month'] = train_set['item_cnt_month'].fillna(0).astype(np.float16)
train_set['item_cnt_month'] = train_set['item_cnt_month'].clip(0, 30)
train_set.head(20)

**Распределение целевой переменной после нормализации**

In [ ]:
sales_train.boxplot(column = ['item_cnt_day'], grid = True, figsize = (10, 10))

In [ ]:
sales_train.item_cnt_day.hist()

In [ ]:
test_sales_train = sales_train
test_sales_train['item_cnt_day'] = test_sales_train['item_cnt_day'].clip(0, 2)
test_sales_train.boxplot(column = ['item_cnt_day'], grid = True, figsize = (10, 10))

In [ ]:
train_set.boxplot(column = ['item_cnt_month'], grid = True, figsize = (10, 10))

In [ ]:
train_set.boxplot(column = ['item_cnt_month'], grid = True, figsize = (10, 10))

**Cоединяем полученный датафрейм со всеми комбинациями с датафреймом, содержащим целевое значение item_cnt_month и признак средней цены. Используем left outer join (язык sql), что означает, что ключевыми столбцами будут являться столбцы датафрейма со всеми комбинациями (grid), отсутствующие значение item_cnt_month будут заполняться 0.**

In [ ]:
train_set = pd.merge(grid, train_set, how = 'left', on = index_cols)
train_set.item_cnt_month = train_set.item_cnt_month.fillna(0)

In [ ]:
train_set.head()

**Добавляем id категории товара и сохраняем полученный датафрейм**

In [ ]:
train_set = pd.merge(train_set, items[['item_id', 'item_category_id']], on = 'item_id')
train_set.to_csv('train_set_with_grid.csv')
train_set.head()

# **Предположение 1: количество продаж в тестовой выборке совпадает с количеством продаж в последнем известном месяце**

**Находим количество продаж за последний известный месяц**

In [ ]:
train_subset = train_set[train_set['date_block_num'] == 33].drop(['date_block_num', 'item_price', 'item_category_id'], axis = 1).sort_values(by = ['shop_id', 'item_id']).reset_index(drop = True)
train_subset.head()

**Находим какое значение примет целевой показатель в тестовой выборке**

In [ ]:
merged = test.merge(train_subset, on = ['shop_id', 'item_id'], how = 'left')[['ID', 'item_cnt_month']]
merged.isna().sum()

**Установим первый бэйслайн без обучения модели и предсказания**

In [ ]:
merged['item_cnt_month'] = merged.item_cnt_month.fillna(0)
submission = merged.set_index('ID')
submission.to_csv('benchmark.csv')

# **Построение признаков**

In [ ]:
baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num', 'item_cnt_month']
train = train_set[baseline_features]
# В качестве индекса установим shop_id
train = train.set_index('shop_id')
# Приведем целевой признак (с типом float64) к типу int
train.item_cnt_month = train.item_cnt_month.astype(int)
# Сохраним train в файл
train.to_csv('train.csv')

In [ ]:
data_set = loadtxt('train.csv', delimiter = ',', skiprows = 1, dtype = int)
train_x = data_set[:, 0:4]
train_y = data_set[:, 4]
test_df = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
# Добавляем в test_df недостающие колонки
merged_test = pd.merge(test_df, items, on = ['item_id'])[['shop_id', 'item_id', 'item_category_id']]
# Установим месяц = 33
merged_test['date_block_num'] = 33
merged_test.set_index('shop_id')
merged_test.head()

# **Простое обучение модели XGBRegressor**

In [ ]:
%%time
model = xgb.XGBRegressor(max_depth = 10, min_child_weight = 0.5, subsample = 1, eta = 0.3, num_round = 1000, seed = 1)
model.fit(train_x, train_y, eval_metric = 'rmse')
preds = model.predict(merged_test.values)
df = pd.DataFrame(preds, columns = ['item_cnt_month'])
df['ID'] = df.index
df = df.set_index('ID')
df.to_csv('simple_xgb.csv')

# **Работа с признаками**

In [ ]:
new_features = []
tqdm.pandas()

In [ ]:
%%time
train_set = pd.read_csv('train_set_with_grid.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
train_set.drop(['Unnamed: 0'], axis = 1, inplace = True)
test_set = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

# Добавим id_category в test_set
test_set = test_set.merge(items[['item_id', 'item_category_id']], on = 'item_id', how = 'left')
# Установим номер месяца в test_set 34
test_set['date_block_num'] = 34
# Для того, чтобы можно было соединить train и test, необходимо сделать равным количество колонок, 
# поэтому установим целевую переменную на тесте в значение -1
test_set['item_cnt_month'] = 0
train_test_set = pd.concat([train_set, test_set], axis = 0)

In [ ]:
train_test_set.head()

In [ ]:
train_test_set = train_test_set.merge(item_ctgrs[['item_category_id', 'type_category_id']], on = 'item_category_id', how = 'left')
_ = train_test_set.drop(['item_category_id'], axis = 1, inplace = True)
train_test_set = train_test_set.rename(columns = {'type_category_id': 'item_category_id'})

_ = item_ctgrs.drop(['item_category_id'], axis = 1, inplace = True)
_ = item_ctgrs.drop(['item_category_name'], axis = 1, inplace = True)

item_ctgrs = item_ctgrs.rename(columns = {'type_category_id': 'item_category_id'})
item_ctgrs = item_ctgrs.rename(columns = {'type_category_name': 'item_category_name'})
item_ctgrs = item_ctgrs.drop_duplicates()
item_ctgrs.index = np.arange(0, len(item_ctgrs))

> # **FE1: Представим сумму продаж (конкретного товара в конкретном магазине) в месяц в качестве признака**

In [ ]:
# for diff in tqdm(range(12)):
#     feature_name = 'prev_shopitem_sales_' + str(diff)
#     train_test_set_2 = train_test_set.copy()
#     train_test_set_2.loc[:, 'date_block_num'] += diff
#     train_test_set_2.rename(columns={'item_cnt_month': feature_name}, inplace=True)
#     train_test_set = train_test_set.merge(train_test_set_2[['shop_id', 'item_id', 'date_block_num', feature_name]], on = ['shop_id', 'item_id', 'date_block_num'], how = 'left')
#     train_test_set[feature_name].fillna(0, inplace = True)
#     new_features.append(feature_name)
train_test_set.head(3)

> # **FE2: Представим среднее число продажи товара в месяц в качестве признака**

In [ ]:
groups = train_test_set.groupby(by = ['item_id', 'date_block_num'])
for diff in tqdm(range(12)):
    feature_name = 'prev_item_sales_' + str(diff)
    result = groups.agg({'item_cnt_month':'mean'})
    result = result.reset_index()
    result.loc[:, 'date_block_num'] += diff
    result.rename(columns={'item_cnt_month': feature_name}, inplace=True)
    train_test_set = train_test_set.merge(result, on = ['item_id', 'date_block_num'], how = 'left')
    train_test_set[feature_name] = train_test_set[feature_name].fillna(0)
    new_features.append(feature_name)        
train_test_set.head(3)

In [ ]:
train_test_set.to_csv('train_test_set.csv')

In [ ]:
for i in new_features:
    print(i)

# **Crossvalidation**

In [ ]:
%%time
baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num'] +  new_features + ['item_cnt_month']

train_test_set['item_cnt_month'] = train_test_set.item_cnt_month.fillna(0).clip(0,20)

# train: отберем строки date_block_num в диапазоне от 0 до 32
train_time_range_lo = (train_test_set['date_block_num'] >= 0)
train_time_range_hi =  (train_test_set['date_block_num'] <= 32)

validation_time =  (train_test_set['date_block_num'] == 33)

test_time =  (train_test_set['date_block_num'] == 34)


# Получаем строки для  train_set, val_set, test_set
cv_trainset = train_test_set[train_time_range_lo & train_time_range_hi]
cv_valset = train_test_set[validation_time]
cv_trainset = cv_trainset[baseline_features]
cv_valset = cv_valset[baseline_features]
testset = train_test_set[test_time]
testset = testset[baseline_features]

# Подготовка nparrays для training/val/test
cv_trainset_vals = cv_trainset.values.astype(int)
trainx = cv_trainset_vals[:, 0:len(baseline_features) - 1]
trainy = cv_trainset_vals[:, len(baseline_features) - 1]

cv_valset_vals = cv_valset.values.astype(int)
valx = cv_valset_vals[:, 0:len(baseline_features) - 1]
valy = cv_valset_vals[:, len(baseline_features) - 1]

testset_vals = testset.values.astype(int)
testx = testset_vals[:, 0:len(baseline_features) - 1]

print('Fitting...')
model = xgb.XGBRegressor(max_depth = 11, min_child_weight=0.5, subsample = 1, eta = 0.3, num_round = 1000, seed = 1, nthread = 16)
model.fit(trainx, trainy, eval_metric='rmse')


preds = model.predict(valx)
preds = np.clip(preds, 0,20)
print('val set rmse: ', sqrt(mean_squared_error(valy, preds)))

preds = model.predict(testx)
preds = np.clip(preds, 0,20)
df = pd.DataFrame(preds, columns = ['item_cnt_month'])
df['ID'] = df.index
df = df.set_index('ID')
df.to_csv('submission.csv')

**В процессе работы я столкнулась с некоторой трудностью, которая заключалась в том, что размер оперативной памяти на Kaggle составляет 16GB максимум. После формирования двух типов фич я не могла обучить модель, используя оба типа фич, так как на этапе кросс-валидации у меня заканчивалась оперативка и сессия в Kaggle вылетала. В итоге я пришла к тому, что можно отдельно обучать модели на каждом типе фичи. В результате лучшей моделью получилась первая, так как ее RMSE был ниже.**